In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
from random import shuffle
from tqdm import tqdm


In [154]:
NUM_FRAME = 10
IMG_SIZE = 100

LR = 0.0001
VIOLENCE_DIR = './Hockey_fights/Violence/'
NON_VIOLENCE_DIR = './Hockey_fights/Non-violence/'


Extracting Dataframes

In [11]:
count = 0
def extract_frames(folder):
    count = 0
    for files in os.listdir(folder):
        cap = cv2.VideoCapture(os.path.join(folder, files))
        sucess, image = cap.read()
        sucess = True
        while sucess:
            sucess, image = cap.read()
            cv2.imwrite("./Hockey_fights/Dataframes/"+ str(count) + ".jpg", image)

            if cv2.waitKey(10) == 27:
                break
            count += 1






In [18]:
count = 0
for files in os.listdir(VIOLENCE_DIR):
    cap = cv2.VideoCapture(os.path.join(VIOLENCE_DIR, files))
    sucess, image = cap.read()
    print(sucess)
    sucess = True
    while sucess:
        sucess, image = cap.read()
        if not sucess:
            break 
        cv2.imwrite("./Hockey_fights/Dataframes/"+ str(count) + ".jpg", image)

        if cv2.waitKey(10) == 27:
            break
        count += 1

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [20]:
print(count)

20057


In [21]:
for files in os.listdir(NON_VIOLENCE_DIR):
    cap = cv2.VideoCapture(os.path.join(NON_VIOLENCE_DIR, files))
    sucess, image = cap.read()
    print(sucess)
    sucess = True
    while sucess:
        sucess, image = cap.read()
        if not sucess:
            break 
        cv2.imwrite("./Hockey_fights/Dataframes/"+ str(count) + ".jpg", image)

        if cv2.waitKey(10) == 27:
            break
        count += 1

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [22]:
print(count)

40056


In [3]:
def create_dataset():
    dataset = []
    image = []
    limit = 0
    c = 0

    for file in tqdm(os.listdir('./Hockey_fights/Dataframes/')):
        path = os.path.join('./Hockey_fights/Dataframes/', file)
        img = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        image.append(np.array(img))
        limit += 1
        c += 1
        if c == NUM_FRAME:
            c = 0
            if limit < 20057:
                dataset.append([image, 1])
            elif limit >= 20056:
                dataset.append([image, 0])
            image = []
    
    shuffle(dataset)
    np.save('dataset.npy', dataset)
    return dataset



In [4]:
data = create_dataset()

100%|██████████| 40056/40056 [01:30<00:00, 444.32it/s]
C:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [5]:
train = data[:-400]
test = data[-400:]

Creating the Model

In [194]:
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TimeDistributed,Dropout, Activation, Flatten,Conv2D, MaxPooling2D,LSTM,Bidirectional, Reshape
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import Input

inputs = Input(shape=[10, IMG_SIZE, IMG_SIZE, 3], name= "input")
convrnn = Conv2D(64, (3, 3), activation='relu', padding = 'same')(inputs)
convrnn = Conv2D(64, (3, 3), activation='relu')(convrnn)
print(convrnn[0][0].shape)
convrnn = MaxPooling2D((2,2))(convrnn[0])
print(convrnn.shape)
convrnn = Conv2D(64, (3, 3), activation='relu')(convrnn)
print(convrnn.shape)
convrnn = MaxPooling2D((2,2))(convrnn)
print(convrnn.shape)
convrnn = Conv2D(64, (3, 3), activation='relu')(convrnn)
convrnn = MaxPooling2D((2,2))(convrnn)
print(convrnn.shape)
convrnn = Flatten()(convrnn)
print(convrnn.shape)

convrnn = tf.reshape(convrnn, (1, 10, 6400))
# print(convrnn.shape)

lstm_fw = LSTM(units=32)
lstm_bw = LSTM(units=32, go_backwards=True)
bi_lstm = Bidirectional(lstm_fw, backward_layer=lstm_bw)(convrnn)

convrnn = Dense(64, activation= 'relu')(bi_lstm)
convrnn = Dense(32, activation= 'relu')(convrnn)
output = Dense(1, activation='sigmoid')(convrnn)

model = tf.keras.Model(inputs = inputs, outputs = output)
model.summary()

(98, 98, 64)
(10, 49, 49, 64)
(10, 47, 47, 64)
(10, 23, 23, 64)
(10, 10, 10, 64)
(10, 6400)
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 10, 100, 100, 3)] 0         
_________________________________________________________________
conv2d_177 (Conv2D)          (None, 10, 100, 100, 64)  1792      
_________________________________________________________________
conv2d_178 (Conv2D)          (None, 10, 98, 98, 64)    36928     
_________________________________________________________________
tf.__operators__.getitem_241 (10, 98, 98, 64)          0         
_________________________________________________________________
max_pooling2d_177 (MaxPoolin (10, 49, 49, 64)          0         
_________________________________________________________________
conv2d_179 (Conv2D)          (10, 47, 47, 64)          36928     
________________________________

In [171]:
X = np.array([i[0] for i in train]).reshape(-1, 10, IMG_SIZE, IMG_SIZE, 3)
Y = np.array([i[1] for i in train])

x_valid = np.array([i[0] for i in test]).reshape(-1, 10, IMG_SIZE, IMG_SIZE, 3)
y_valid = np.array([i[1] for i in test])

In [172]:

X = X.astype('float32')/255
x_valid = x_valid.astype('float32')/255
Y = Y.astype('float32').reshape((-1, 1))
y_valid = y_valid.astype('float32').reshape((-1, 1))



In [173]:
print(Y.shape)
print(y_valid.shape)
print(X.shape)
print(x_valid.shape)

(3605, 1)
(400, 1)
(3605, 10, 100, 100, 3)
(400, 10, 100, 100, 3)


In [192]:
x_try = X[:20]
y_try = Y[:20]
print(len(x_try), len(y_try))

20 20


In [195]:
opt = tf.keras.optimizers.Adam(learning_rate= LR)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

In [196]:
model.fit(X, Y)

113/113 [==============================] - 565s 5s/step - loss: 0.6954 - accuracy: 0.4913
